In [0]:
%run "../includes/configuration"

In [0]:
race_results_df = spark.read.parquet(f"{presentation_folder_path}/race_results")

In [0]:
display(race_results_df)

In [0]:
from pyspark.sql.functions import sum, when, col, count

driver_standing_df = race_results_df \
    .groupBy("race_year", "driver_name", "driver_nationality", "team") \
    .agg(sum("points").alias("total_points"), count(when(col('position') == 1, True)).alias("wins"))

In [0]:
display(driver_standing_df.filter("race_year = 2020"))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

driver_rank_sepc = Window.partitionBy('race_year').orderBy(desc('total_points'), desc('wins'))
final_df = driver_standing_df.withColumn('rank', rank().over(driver_rank_sepc))

In [0]:
display(final_df.filter("race_year = 2020"))


In [0]:
#final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/driver_standings")

In [0]:
%sql
USE CATALOG hive_metastore;

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.driver_standings")